In [2]:
# Libraries
import json
import spacy
from transformers import pipeline
from collections import defaultdict

In [10]:
# Load spaCy model
nlp = spacy.load("de_core_news_sm")  # Use a German spaCy model

# Load data from JSON file
with open("chunks.json", "r", encoding="utf-8") as file:
    data_list = json.load(file)

# Generate dataset
dataset = []

# Process each entry in the JSON file
for data in data_list:
    # Assuming the JSON structure: {"id": ..., "profession": ..., "text": ...}
    profession = data.get("profession", "")
    text = data.get("text", "")

    # Change Einstiegsgehalt lt. KV: to Einstiegsgehalt lt. KV -
    text = text.replace("Einstiegsgehalt lt. KV:\n", "Einstiegsgehalt lt. KV - ")
    #Same for BMS and BHS
    text = text.replace("BMS:\n", "BMS - ")
    text = text.replace("BHS:\n", "BHS - ")
    # # Remove Einstiegsgehalt lt. KV: and BMS: from all texts
    # text = text.replace("Einstiegsgehalt lt. KV:", "")
    # text = text.replace("BMS:", "")

    # Tokenize and extract sections using spaCy (assuming sections are separated by '\n')
    sections = [section.strip() for section in text.split('\n') if section.strip()]

    # Combine sections that logically belong together
    combined_sections = []
    current_section = ""

    for section in sections:
        if current_section and current_section.endswith(":"):
            # Combine with the next section
            combined_sections.append(current_section + " " + section)
            current_section = ""
        else:
            # Start a new section
            combined_sections.append(current_section)
            current_section = section

    # Generate contextually meaningful questions for each combined section
    for i, combined_section in enumerate(combined_sections):
        combined_section_list = []
        # Customize this logic based on your specific content
        if "Ausbildungsform:" in combined_section:
            question = f"Welche Ausbildung braucht ein/eine {profession}?"
            #dataset.append({"question": question, "answer": combined_section})
        elif "Gehalt:" in combined_section:
            question = f"Was ist das Gehalt eines/einer {profession}?"
            #dataset.append({"question": question, "answer": combined_section})
        elif "Berufsbereiche:" in combined_section:
            question = f"In welchen Berufsbereichen arbeitet ein/eine {profession}?"
            #dataset.append({"question": question, "answer": combined_section})
        elif combined_section != profession and combined_section != "Zurück":
            combined_section_list.append(combined_section)
            try:
                combined_section = ', '.join(combined_section_list)
            except:
                combined_section = combined_section_list[0]
            question = f"Was macht ein/eine {profession}?"
        else:
            continue
        dataset.append({"question": question, "answer": combined_section})

# Save the dataset to a JSON file
with open("output_dataset_7.json", "w", encoding="utf-8") as output_file:
    json.dump(dataset, output_file, ensure_ascii=False, indent=2)

# print("Dataset generation complete.")
dataset


[{'question': 'Was macht ein/eine 24-Stunden-BetreuerIn?', 'answer': ''},
 {'question': 'In welchen Berufsbereichen arbeitet ein/eine 24-Stunden-BetreuerIn?',
  'answer': 'Berufsbereiche: Soziales, Gesundheit, Schönheitspflege'},
 {'question': 'Was macht ein/eine 24-Stunden-BetreuerIn?', 'answer': ''},
 {'question': 'Welche Ausbildung braucht ein/eine 24-Stunden-BetreuerIn?',
  'answer': 'Ausbildungsform: BMS/BHS'},
 {'question': 'Was ist das Gehalt eines/einer 24-Stunden-BetreuerIn?',
  'answer': 'Einstiegsgehalt lt. KV - Gehalt: BMS - € 2.050,- bis € 2.230,- *'},
 {'question': 'Was macht ein/eine 24-Stunden-BetreuerIn?', 'answer': ''},
 {'question': 'Was macht ein/eine 24-Stunden-BetreuerIn?',
  'answer': '24-Stunden-BetreuerInnen unterstützen pflegebedürftige Menschen, die eine permanente Betreuung benötigen, bei diesen zu Hause. Sie leisten Hilfe bei der täglichen Körperpflege und Hygiene, z.B. beim An- und Auskleiden, Waschen und Duschen, sowie bei der Nahrungsaufnahme und Medikam

In [11]:
# Load the generated dataset from output_dataset.json
with open("output_dataset_7.json", "r", encoding="utf-8") as file:
    dataset = json.load(file)
    
# Create a defaultdict to store answers for each question
grouped_answers = defaultdict(list)

# Iterate through the dataset and group answers by question
for entry in dataset:
    if entry["answer"] == "Einstiegsgehalt: Gehalt:":
        entry["answer"] = "Zu diesem Beruf ist leider kein Gehalt angeführt"
    question = entry["question"]
    answer = entry["answer"]
    grouped_answers[question].append(answer)

# Create a new list to store combined entries
combined_dataset = []

# Iterate through grouped answers and create combined entries
for question, answers in grouped_answers.items():
    combined_entry = {"question": question, "combined_answers": " ".join(answers)}
    combined_entry["combined_answers"] = combined_entry["combined_answers"].strip()
    combined_dataset.append(combined_entry)

# Save the combined dataset to a new JSON file
with open("combined_questions_4.json", "w", encoding="utf-8") as output_file:
    json.dump(combined_dataset, output_file, ensure_ascii=False, indent=2)

print("Combining answers complete.")
combined_dataset

Combining answers complete.


[{'question': 'Was macht ein/eine 24-Stunden-BetreuerIn?',
  'combined_answers': '24-Stunden-BetreuerInnen unterstützen pflegebedürftige Menschen, die eine permanente Betreuung benötigen, bei diesen zu Hause. Sie leisten Hilfe bei der täglichen Körperpflege und Hygiene, z.B. beim An- und Auskleiden, Waschen und Duschen, sowie bei der Nahrungsaufnahme und Medikamenteneinnahme. Ein wichtiger Teil sind soziale Aktivitäten und die aktivierende Betreuung: Gesellschaft leisten, die betreute Person strukturiert durch den Tag führen, mit ihr Spaziergänge und kurze Ausflüge machen, auf Wunsch Geschichten vorlesen und Ähnliches. Weiters sind 24-Stunden-BetreuerInnen für die Hauswirtschaft und Einkäufe zuständig. Sie stellen die Versorgung mit Lebensmitteln sicher, räumen auf, waschen das Geschirr ab, kümmern sich um die Wäsche und bügeln. Auch leichte Gartenarbeiten können nach Absprache erledigt werden. Schließlich übernehmen die BetreuerInnen auch die Nachtpflege: Falls nötig betreuen sie die 